#### Food Inspection Data Set Description
"The Health Division of the Department of Inspectional Services ensures that all food establishments in the City of Boston meet relevant sanitary codes and standards. Businesses that serve food are inspected at least once a year, and follow-up inspections are performed on high risk establishments. Health inspections are also conducted in response to complaints of unsanitary conditions or illness."  *(Source - https://data.boston.gov)*

The number of violations is defined as:

1) *      Minor Violation

2) **     Major Violation

3) ***    Severe Violation

*(Source - https://restaurantprediction.weebly.com)*


#### Data Cleaning Summary
1) Convert attributes to appropriate date type and format

    a) licenseno and property_id to object from int and float, respectively
    
    b) all -dttm columns (5) to datetime 
    
    c) zip to 5-digit format
    
2) Filter data by "active" license

3) Filter data for inspection result ('resultdttm') for dates from 2015 to current (to match df_crime)

4) Take care of missing values - **TO BE DONE**

    a) there are 21 rows with zip as 00000.  These 21 rows are from Boston, MA and of two business names.  Potentially can fill zip with appropriate zip code
    
    b) do not necessarily need to delete any other rows.

#### Current Final Clean Data Set Information

1) Name = dfins_activefrm15

2) Shape = 171,293 rows x 26 columns

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
pd.set_option('display.max_columns', None)  # display all columns on screen
import re

In [2]:
df_ins_org = pd.read_csv(r'C:\Users\ale\Desktop\MIST6150\Project\df_ins.csv')

C:\Users\ale\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_ins_org.head(3)

,businessname,dbaname,legalowner,namelast,namefirst,licenseno,issdttm,expdttm,licstatus,licensecat,descript,result,resultdttm,violation,viollevel,violdesc,violdttm,violstatus,statusdate,comments,address,city,state,zip,property_id,location
0,100 Percent Delicia Food,NaN,BRENNAN PATRICK E,Marte,Civelis,87059,2013-04-05 12:47:23,2019-12-31 23:59:00,Active,FS,Eating & Drinking,HE_Fail,2013-02-15 12:19:42,15-4-202.16,*,Non-Food Contact Surfaces,2013-02-15 12:19:42,Fail,NaN,Provide glass storage rack.,635 Hyde Park AVE,Roslindale,MA,02131,77476.0,"(42.278590000, -71.119440000)"
1,100 Percent Delicia Food,NaN,BRENNAN PATRICK E,Marte,Civelis,87059,2013-04-05 12:47:23,2019-12-31 23:59:00,Active,FS,Eating & Drinking,HE_Fail,2013-02-15 12:19:42,16-4-501.11/.15,*,Dishwashng Facilities,2013-02-15 12:19:42,Fail,NaN,Provide dish rack over 3 bay sink to replace m...,635 Hyde Park AVE,Roslindale,MA,02131,77476.0,"(42.278590000, -71.119440000)"
2,100 Percent Delicia Food,NaN,BRENNAN PATRICK E,Marte,Civelis,87059,2013-04-05 12:47:23,2019-12-31 23:59:00,Active,FS,Eating & Drinking,HE_Fail,2013-02-15 12:19:42,16-4-501.11/.15,*,Dishwashng Facilities,2013-02-15 12:19:42,Fail,NaN,Provide dish washer at ware washing area.,635 Hyde Park AVE,Roslindale,MA,02131,77476.0,"(42.278590000, -71.119440000)"


In [4]:
df_ins_org.shape

(562720, 26)

In [5]:
df_ins_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562720 entries, 0 to 562719
Data columns (total 26 columns):
businessname    562720 non-null object
dbaname         5426 non-null object
legalowner      392419 non-null object
namelast        562720 non-null object
namefirst       562606 non-null object
licenseno       562720 non-null int64
issdttm         562720 non-null object
expdttm         562720 non-null object
licstatus       562720 non-null object
licensecat      562720 non-null object
descript        562720 non-null object
result          562720 non-null object
resultdttm      562720 non-null object
violation       530897 non-null object
viollevel       530897 non-null object
violdesc        530896 non-null object
violdttm        562720 non-null object
violstatus      530897 non-null object
statusdate      233765 non-null object
comments        562716 non-null object
address         562720 non-null object
city            562694 non-null object
state           562720 non-null ob

In [6]:
# copy df_ins_org as df_ins.  
df_ins = df_ins_org.copy()

In [7]:
# change data type of "issdttm", "expdttm", "resultdttm", "violdttm", and "statusdate" to date/time 
df_ins['issdttm']=pd.to_datetime(df_ins['issdttm'], format='%Y-%m-%d', errors='coerce')
df_ins['expdttm']=pd.to_datetime(df_ins['expdttm'], format='%Y-%m-%d', errors = 'coerce')
df_ins['resultdttm']=pd.to_datetime(df_ins['resultdttm'], format='%Y-%m-%d', errors = 'coerce')
df_ins['violdttm']=pd.to_datetime(df_ins['violdttm'], format='%Y-%m-%d', errors = 'coerce')
df_ins['statusdate']=pd.to_datetime(df_ins['statusdate'], format='%Y-%m-%d', errors = 'coerce')

In [8]:
# change data type of "licenseno" from integer to object, and "property_id" from float to object
df_ins['licenseno']=df_ins['licenseno'].astype(str)
df_ins['property_id']=df_ins['property_id'].astype(str)

In [9]:
# change zip to correct 5-digit format
df_ins['zip']=df_ins['zip'].astype(str).str.zfill(5)

In [10]:
# confirm data type change for df_ins
df_ins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562720 entries, 0 to 562719
Data columns (total 26 columns):
businessname    562720 non-null object
dbaname         5426 non-null object
legalowner      392419 non-null object
namelast        562720 non-null object
namefirst       562606 non-null object
licenseno       562720 non-null object
issdttm         562607 non-null datetime64[ns]
expdttm         562449 non-null datetime64[ns]
licstatus       562720 non-null object
licensecat      562720 non-null object
descript        562720 non-null object
result          562720 non-null object
resultdttm      556323 non-null datetime64[ns]
violation       530897 non-null object
viollevel       530897 non-null object
violdesc        530896 non-null object
violdttm        530894 non-null datetime64[ns]
violstatus      530897 non-null object
statusdate      233765 non-null datetime64[ns]
comments        562716 non-null object
address         562720 non-null object
city            562694 non-null 

In [11]:
# filter by license status = active
df_ins=df_ins[df_ins['licstatus']=='Active']
df_ins.shape

(355029, 26)

In [12]:
# filter by license expirate date = 12/31/2019
df_ins=df_ins[df_ins['expdttm']>='2019']
df_ins.shape

(342409, 26)

In [13]:
# filter by resultdttm=>2015 (to match with df_crime)
df_ins=df_ins[df_ins['resultdttm']>='2015']
df_ins.shape

(164383, 26)

In [14]:
df_ins.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164383 entries, 16 to 562719
Data columns (total 26 columns):
businessname    164383 non-null object
dbaname         951 non-null object
legalowner      108363 non-null object
namelast        164383 non-null object
namefirst       164383 non-null object
licenseno       164383 non-null object
issdttm         164383 non-null datetime64[ns]
expdttm         164383 non-null datetime64[ns]
licstatus       164383 non-null object
licensecat      164383 non-null object
descript        164383 non-null object
result          164383 non-null object
resultdttm      164383 non-null datetime64[ns]
violation       158686 non-null object
viollevel       158686 non-null object
violdesc        158686 non-null object
violdttm        158686 non-null datetime64[ns]
violstatus      158686 non-null object
statusdate      71483 non-null datetime64[ns]
comments        164383 non-null object
address         164383 non-null object
city            164383 non-null o

In [15]:
df_ins.describe()

,businessname,dbaname,legalowner,namelast,namefirst,licenseno,issdttm,expdttm,licstatus,licensecat,descript,result,resultdttm,violation,viollevel,violdesc,violdttm,violstatus,statusdate,comments,address,city,state,zip,property_id,location
count,164383,951,108363,164383,164383,164383,164383,164383,164383,164383,164383,164383,164383,158686,158686,158686,158686,158686,71483,164383,164383,164383,164383,164383,164383,121472
unique,3283,17,1482,2783,1437,3631,3628,1,1,4,4,12,34767,90,3,88,29090,2,9308,73776,2740,45,3,36,2377,1971
top,Dunkin Donuts,1844 Inc.,,CVS PHARMACY INC.,,137896,2015-06-30 08:46:46,2019-12-31 23:59:00,Active,FS,Eating & Drinking,HE_Fail,2015-03-04 10:59:57,23-4-602.13,*,Non-Food Contact Surfaces Clean,2015-03-20 11:02:51,Fail,2017-06-30 15:24:18,,1 Citywide ST,Boston,MA,02116,nan,"(42.285670000, -71.155480000)"
freq,897,177,12227,1052,71191,415,415,164383,164383,83376,83376,71835,50,12635,118735,12635,50,87204,57,7114,2139,64733,162408,16089,26074,576
first,NaN,NaN,NaN,NaN,NaN,NaN,2011-11-02 14:01:20,2019-12-31 23:59:00,NaN,NaN,NaN,NaN,2015-01-02 10:07:21,NaN,NaN,NaN,2007-06-05 14:20:00,NaN,2015-01-02 15:33:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-07 10:40:45,2019-12-31 23:59:00,NaN,NaN,NaN,NaN,2019-03-08 13:54:29,NaN,NaN,NaN,2019-03-08 13:54:29,NaN,2019-03-08 15:42:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##Reviewing attributes with finite sets of values

In [16]:
df_ins['result'].unique()

array(['HE_Fail', 'HE_Pass', 'HE_FailExt', 'HE_Filed', 'HE_Hearing',
       'HE_NotReq', 'HE_OutBus', 'HE_TSOP', 'HE_Closure', 'HE_Misc',
       'HE_FAILNOR', 'Pass'], dtype=object)

In [17]:
df_ins['licensecat'].unique()

array(['FS', 'FT', 'RF', 'MFW'], dtype=object)

In [18]:
df_ins['descript'].unique()

array(['Eating & Drinking', 'Eating & Drinking w/ Take Out',
       'Retail Food', 'Mobile Food Walk On'], dtype=object)

In [19]:
df_ins['violstatus'].unique()

array(['Fail', nan, 'Pass'], dtype=object)

In [20]:
df_ins.groupby('city')['businessname','zip'].count()

,businessname,zip
city,,
,4,4
ALLSTON,938,938
Allston,6403,6403
BOSTON,9663,9663
BRIGHTON,675,675
Back Bay/,12,12
Boston,64733,64733
Brighton,4677,4677
Brighton/,26,26


In [21]:
df_ins['city'] = df_ins['city'].str.replace('/', '')

In [22]:
df_ins['city'] = df_ins['city'].str.upper()

In [23]:
df_ins.groupby('city')['businessname','zip'].count()

,businessname,zip
city,,
,4,4
ALLSTON,7341,7341
BACK BAY,12,12
BOSTON,74396,74396
BRIGHTON,5378,5378
CHARLESTOWN,1804,1804
CHESTNUT HILL,132,132
DORCHESTER,20123,20123
DOWNTOWNFINANCIAL DISTRICT,1,1


In [24]:
# dropping un-needed columns
cols_to_drop=['dbaname','legalowner','namelast','namefirst','descript','state']
df_ins=df_ins.drop(cols_to_drop,axis=1)
df_ins.shape

(164383, 20)

In [25]:
# check for null data
df_ins.isnull().sum()

businessname        0
licenseno           0
issdttm             0
expdttm             0
licstatus           0
licensecat          0
result              0
resultdttm          0
violation        5697
viollevel        5697
violdesc         5697
violdttm         5697
violstatus       5697
statusdate      92900
comments            0
address             0
city                0
zip                 0
property_id         0
location        42911
dtype: int64

In [26]:
# entries with zip = 0000
df_ins[df_ins['zip']<'02108']

,businessname,licenseno,issdttm,expdttm,licstatus,licensecat,result,resultdttm,violation,viollevel,violdesc,violdttm,violstatus,statusdate,comments,address,city,zip,property_id,location
203955,Frog Pond Boston @ Boston Common,75274,2012-02-24 14:04:35,2019-12-31 23:59:00,Active,FT,HE_Filed,2015-06-30 11:39:06,23-4-602.13,*,Non-Food Contact Surfaces Clean,2015-06-30 11:39:06,Fail,NaT,clean drain compartments to fryers. Clean inte...,,,0000,nan,NaN
203956,Frog Pond Boston @ Boston Common,75274,2012-02-24 14:04:35,2019-12-31 23:59:00,Active,FT,HE_Filed,2018-06-29 10:17:36,23-4-602.13,*,Non-Food Contact Surfaces Clean,2018-06-29 10:17:36,Fail,NaT,Clean to remove ice build up from dippin dots ...,,,0000,nan,NaN
203969,Frog Pond Boston @ Boston Common,75274,2012-02-24 14:04:35,2019-12-31 23:59:00,Active,FT,HE_Pass,2016-12-02 12:42:09,NaN,NaN,NaN,NaT,NaN,NaT,,,,0000,nan,NaN
203970,Frog Pond Boston @ Boston Common,75274,2012-02-24 14:04:35,2019-12-31 23:59:00,Active,FT,HE_Pass,2017-07-14 12:50:14,NaN,NaN,NaN,NaT,NaN,NaT,,,,0000,nan,NaN
280679,LEGAL SEA FOOD,25105,2012-01-11 07:50:32,2019-12-31 23:59:00,Active,MFW,HE_Fail,2018-06-07 08:27:17,NaN,NaN,NaN,NaT,NaN,NaT,,CITYWIDE,BOSTON,0000,nan,NaN
280680,LEGAL SEA FOOD,25105,2012-01-11 07:50:32,2019-12-31 23:59:00,Active,MFW,HE_Fail,2018-06-07 08:40:39,05-4-302.12,*,Food Thermometers Provided,2018-06-07 08:40:39,Fail,NaT,provide accurate cooks thermometer,CITYWIDE,BOSTON,0000,nan,NaN
280681,LEGAL SEA FOOD,25105,2012-01-11 07:50:32,2019-12-31 23:59:00,Active,MFW,HE_Fail,2018-06-07 08:40:39,14-4-202.11,*,Food Contact Surfaces Design,2018-06-07 08:40:39,Fail,NaT,provide extra serving utensils,CITYWIDE,BOSTON,0000,nan,NaN
280682,LEGAL SEA FOOD,25105,2012-01-11 07:50:32,2019-12-31 23:59:00,Active,MFW,HE_Fail,2018-06-07 08:40:39,17-4-302.14,*,Test Kit Provided,2018-06-07 08:40:39,Fail,NaT,provide proper test kit,CITYWIDE,BOSTON,0000,nan,NaN
280683,LEGAL SEA FOOD,25105,2012-01-11 07:50:32,2019-12-31 23:59:00,Active,MFW,HE_Fail,2018-06-07 08:40:39,27-5-103.11-.12,***,Hot and Cold Water,2018-06-07 08:40:39,Fail,NaT,repair handsink to provide hot and cold water ...,CITYWIDE,BOSTON,0000,nan,NaN
280684,LEGAL SEA FOOD,25105,2012-01-11 07:50:32,2019-12-31 23:59:00,Active,MFW,HE_Fail,2018-06-07 08:40:39,29-5-201/02.11,*,Installed and Maintained,2018-06-07 08:40:39,Fail,NaT,repair leak from under steamtable,CITYWIDE,BOSTON,0000,nan,NaN


In [27]:
# there are 35 unique zip codes (minus 0000)
df_ins.groupby(['zip']).count()

,businessname,licenseno,issdttm,expdttm,licstatus,licensecat,result,resultdttm,violation,viollevel,violdesc,violdttm,violstatus,statusdate,comments,address,city,property_id,location
zip,,,,,,,,,,,,,,,,,,,
0000,21,21,21,21,21,21,21,21,16,16,16,16,16,7,21,21,21,21,0
02108,5221,5221,5221,5221,5221,5221,5221,5221,4998,4998,4998,4998,4998,2303,5221,5221,5221,5221,4499
02109,6177,6177,6177,6177,6177,6177,6177,6177,5946,5946,5946,5946,5946,2761,6177,6177,6177,6177,3175
02110,4535,4535,4535,4535,4535,4535,4535,4535,4179,4179,4179,4179,4179,1898,4535,4535,4535,4535,3515
02111,5046,5046,5046,5046,5046,5046,5046,5046,4675,4675,4675,4675,4675,2214,5046,5046,5046,5046,4441
02113,3615,3615,3615,3615,3615,3615,3615,3615,3496,3496,3496,3496,3496,1577,3615,3615,3615,3615,3230
02114,6582,6582,6582,6582,6582,6582,6582,6582,6182,6182,6182,6182,6182,2827,6582,6582,6582,6582,5181
02115,9977,9977,9977,9977,9977,9977,9977,9977,9494,9494,9494,9494,9494,4314,9977,9977,9977,9977,7253
02116,16089,16089,16089,16089,16089,16089,16089,16089,15687,15687,15687,15687,15687,6960,16089,16089,16089,16089,13953


In [28]:
df_ins.groupby('licensecat').count()

,businessname,licenseno,issdttm,expdttm,licstatus,result,resultdttm,violation,viollevel,violdesc,violdttm,violstatus,statusdate,comments,address,city,zip,property_id,location
licensecat,,,,,,,,,,,,,,,,,,,
FS,83376,83376,83376,83376,83376,83376,83376,81173,81173,81173,81173,81173,36936,83376,83376,83376,83376,83376,62663
FT,60609,60609,60609,60609,60609,60609,60609,58477,58477,58477,58477,58477,26370,60609,60609,60609,60609,60609,45605
MFW,2258,2258,2258,2258,2258,2258,2258,2083,2083,2083,2083,2083,926,2258,2258,2258,2258,2258,97
RF,18140,18140,18140,18140,18140,18140,18140,16953,16953,16953,16953,16953,7251,18140,18140,18140,18140,18140,13107


In [29]:
df_ins.groupby(['viollevel']).count()

,businessname,licenseno,issdttm,expdttm,licstatus,licensecat,result,resultdttm,violation,violdesc,violdttm,violstatus,statusdate,comments,address,city,zip,property_id,location
viollevel,,,,,,,,,,,,,,,,,,,
*,118735,118735,118735,118735,118735,118735,118735,118735,118735,118735,118735,118735,52138,118735,118735,118735,118735,118735,88626
**,10191,10191,10191,10191,10191,10191,10191,10191,10191,10191,10191,10191,4920,10191,10191,10191,10191,10191,7691
***,29760,29760,29760,29760,29760,29760,29760,29760,29760,29760,29760,29760,14425,29760,29760,29760,29760,29760,21780


In [30]:
df_ins.head()

,businessname,licenseno,issdttm,expdttm,licstatus,licensecat,result,resultdttm,violation,viollevel,violdesc,violdttm,violstatus,statusdate,comments,address,city,zip,property_id,location
16,100 Percent Delicia Food,87059,2013-04-05 12:47:23,2019-12-31 23:59:00,Active,FS,HE_Fail,2015-07-27 14:22:38,08-3-302.11/3-304.11,***,Separation Segregation Cross Contamination,2015-07-27 14:22:38,Fail,NaT,Observed raw chicken stored over ready to eat ...,635 Hyde Park AVE,ROSLINDALE,02131,77476.0,"(42.278590000, -71.119440000)"
17,100 Percent Delicia Food,87059,2013-04-05 12:47:23,2019-12-31 23:59:00,Active,FS,HE_Fail,2015-07-27 14:22:38,08-3-305-307.11,*,Food Protection,2015-07-27 14:22:38,Fail,NaT,Closet next to microwave with pots stored on t...,635 Hyde Park AVE,ROSLINDALE,02131,77476.0,"(42.278590000, -71.119440000)"
18,100 Percent Delicia Food,87059,2013-04-05 12:47:23,2019-12-31 23:59:00,Active,FS,HE_Fail,2015-07-27 14:22:38,08-3-305-307.11,*,Food Protection,2015-07-27 14:22:38,Fail,NaT,Observed buckets of food products stored direc...,635 Hyde Park AVE,ROSLINDALE,02131,77476.0,"(42.278590000, -71.119440000)"
19,100 Percent Delicia Food,87059,2013-04-05 12:47:23,2019-12-31 23:59:00,Active,FS,HE_Fail,2015-07-27 14:22:38,17-4-204.112/.115,*,Equipment Thermometers,2015-07-27 14:22:38,Fail,NaT,Provide working internal thermometers for all ...,635 Hyde Park AVE,ROSLINDALE,02131,77476.0,"(42.278590000, -71.119440000)"
20,Ariana Restaurant,136868,2015-08-24 11:29:01,2019-12-31 23:59:00,Active,FS,HE_Pass,2017-10-17 19:37:12,NaN,NaN,NaN,NaT,NaN,NaT,,123 Antwerp ST,BRIGHTON,02135,343305.0,NaN


In [34]:
df_ins.violstatus.unique()

array(['Fail', nan, 'Pass'], dtype=object)

In [38]:
df_ins.groupby('viollevel').value_counts(dropna=False)

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_counts'